#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [29]:
filepath = os.getcwd() + '/event_data'
file_path_list = glob.glob(os.path.join(filepath,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [32]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [33]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nitesh_nosql_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
try:
    session.set_keyspace('nitesh_nosql_project')
except Exception as e:
    print(e)

### TAKS-1 

**Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**

**Creating Table**

Since we need to select item by session_id and itemInSession, they form primary key Where session_id is partition key and itemInSession is clustering column.

In [35]:
query = "CREATE TABLE IF NOT EXISTS songplays_by_session_id_item_in_session"
query = query + "(session_id int, item_in_session int, artist text, song text, length int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

**Inserting records**

In [36]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_by_session_id_item_in_session (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], int(float(line[5])) ))

#### Result:

In [37]:
query = "select artist, song, length from songplays_by_session_id_item_in_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)


Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495)


### TAKS-2

**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

**Creating Table**

Since we need to select item by user_id and session_id, they form partition key and since we need to sort items by itemInsessions, it forms clustering column. 

In [38]:
query = "CREATE TABLE IF NOT EXISTS songsplays_by_userid_session_id"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

**Inserting records**

In [39]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songsplays_by_userid_session_id (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))

#### Result:

In [40]:
query = "select artist, song, first_name, last_name  from songsplays_by_userid_session_id WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)


Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### TAKS-3

**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

Since we need to select item by song name, it forms partition key, I added user_id as clustering column to get the result in sorted order 

**Creating Table**

In [45]:
query = "CREATE TABLE IF NOT EXISTS songplays_by_username"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

**Inserting records**

In [46]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_by_username (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, ( line[9],int(line[10]), line[1], line[4]))

#### Result:

In [47]:
query = "select first_name, last_name  from songplays_by_username WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Droping the tables

In [48]:
query = "drop table songplays_by_session_id_item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songsplays_by_userid_session_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songplays_by_username"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Closing the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()